In [1]:
import json
import requests
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize 


load_dotenv(verbose=True)

True

In [2]:
api_key = os.getenv("CRUNCHBASE_API_KEY") # extract API key 
base_url = "https://api.crunchbase.com/api/"
params = {"user_key" : api_key}

In [75]:
query_swiss = {
    "field_ids": [
        "name",
        #"created_at",
        "short_description",
        "location_identifiers",
        #"funding_stage",
        #"funding_total",
        "categories",
        "category_groups",
        "facet_ids",
        "twitter",
        "rank_org"
        
    ],
    "limit": 1000,
    "query": [
        {
          "type": "predicate",
          "field_id": "location_identifiers",
          "operator_id": "includes",
          "values": ["Switzerland"]
        },
    ]
}

r = requests.post(f"https://api.crunchbase.com/api/v4/searches/organizations", json=query_swiss, params=params)

In [84]:
query = {
    "field_ids": [
        "name",
        "short_description",
        "location_identifiers",
        "categories",
        "category_groups",
        "facet_ids",
        "twitter",
        "rank_org"
        
    ],
    "limit": 1000,
}

r = requests.post(f"https://api.crunchbase.com/api/v4/searches/organizations", json=query, params=params)

In [85]:
json_data = json.loads(r.text)

In [86]:
df = pd.json_normalize(json_data['entities'])
#df.head()

## Data Cleaning

In [87]:
df = df.drop(["uuid", "properties.identifier.image_id", "properties.identifier.uuid", "properties.identifier.entity_def_id"], axis=1) # remove useless columns

In [88]:
if (df["properties.name"] == df["properties.identifier.value"]).all() == True: # properties.identifier.value does not bring new info
    df = df.drop(["properties.identifier.value"], axis=1)

In [94]:
df.sort_values("properties.rank_org").head(10)

,properties.name,properties.facet_ids,properties.identifier.permalink,properties.category_groups,properties.short_description,properties.rank_org,properties.categories,properties.location_identifiers,properties.twitter.value
0,SoftBank,"[investor, company, rank]",softbank,"[{'entity_def_id': 'category_group', 'permalin...","SoftBank provides fixed-line, mobile telephony...",1,"[{'entity_def_id': 'category', 'permalink': 'b...","[{'permalink': 'tokyo-tokyo', 'uuid': '127ba62...",https://twitter.com/softbank_group
1,Flutterwave,"[company, rank]",flutterwave,"[{'entity_def_id': 'category_group', 'permalin...",Flutterwave is a fintech company that provides...,2,"[{'entity_def_id': 'category', 'permalink': 'f...","[{'permalink': 'san-francisco-california', 'uu...",https://www.twitter.com/theflutterwave
2,Klarna,"[investor, company, rank]",klarna,"[{'entity_def_id': 'category_group', 'permalin...",Klarna is an eCommerce payment solutions platf...,3,"[{'entity_def_id': 'category', 'permalink': 'e...","[{'permalink': 'stockholm-stockholms-lan', 'uu...",http://twitter.com/klarna
3,Compass,"[company, rank]",compassinc,"[{'entity_def_id': 'category_group', 'permalin...",Compass is a real estate technology company th...,4,"[{'entity_def_id': 'category', 'permalink': 'p...","[{'permalink': 'new-york-new-york', 'uuid': 'd...",https://twitter.com/Compass
4,Zoom,"[investor, company, rank]",zoom-video-communications,"[{'entity_def_id': 'category_group', 'permalin...",Zoom is a software company that offers a commu...,5,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'san-jose-california', 'uuid': ...",http://twitter.com/zoom_us
5,Crowdcube,"[investor, company, rank]",crowdcube,"[{'entity_def_id': 'category_group', 'permalin...",Crowdcube is an equity crowdfunding platform f...,6,"[{'entity_def_id': 'category', 'permalink': 'b...","[{'permalink': 'exeter-devon', 'uuid': 'c5f8ba...",https://twitter.com/Crowdcube
6,Ant Group,"[investor, company, rank]",ant-group,"[{'entity_def_id': 'category_group', 'permalin...",Ant Group strives to enable all consumers and ...,7,"[{'entity_def_id': 'category', 'permalink': 'b...","[{'permalink': 'hangzhou-zhejiang', 'uuid': '4...",https://twitter.com/antgroup
7,Antler,"[investor, rank]",antler-be9c,"[{'entity_def_id': 'category_group', 'permalin...",Antler is a global early-stage venture capital...,8,"[{'entity_def_id': 'category', 'permalink': 'v...","[{'permalink': 'singapore-central-region', 'uu...",https://twitter.com/AntlerGlobal
8,Intel,"[investor, company, rank]",intel,"[{'entity_def_id': 'category_group', 'permalin...","Intel designs, manufactures, and sells integra...",9,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'santa-clara-california', 'uuid...",http://twitter.com/intel
9,Apollo,"[investor, company, rank]",apollo-global-management-llc,"[{'entity_def_id': 'category_group', 'permalin...",Apollo Global Management is a global asset man...,10,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'new-york-new-york', 'uuid': 'd...",https://twitter.com/apolloglobal


In [90]:
df[["properties.name", "properties.short_description"]].head(10)

,properties.name,properties.short_description
0,SoftBank,"SoftBank provides fixed-line, mobile telephony..."
1,Flutterwave,Flutterwave is a fintech company that provides...
2,Klarna,Klarna is an eCommerce payment solutions platf...
3,Compass,Compass is a real estate technology company th...
4,Zoom,Zoom is a software company that offers a commu...
5,Crowdcube,Crowdcube is an equity crowdfunding platform f...
6,Ant Group,Ant Group strives to enable all consumers and ...
7,Antler,Antler is a global early-stage venture capital...
8,Intel,"Intel designs, manufactures, and sells integra..."
9,Apollo,Apollo Global Management is a global asset man...


In [92]:
df["properties.category_groups"].values[0]

[{'entity_def_id': 'category_group',
  'permalink': 'financial-services-d7b6',
  'uuid': 'd7b67d9f-9b01-3864-1400-64fe3d4bebc3',
  'value': 'Financial Services'},
 {'entity_def_id': 'category_group',
  'permalink': 'hardware-2e6e',
  'uuid': '2e6eafef-f310-ba60-d932-62f866a87779',
  'value': 'Hardware'},
 {'entity_def_id': 'category_group',
  'permalink': 'internet-services',
  'uuid': 'd1079d33-97d7-1f5a-7e6c-b80d5373a3e0',
  'value': 'Internet Services'},
 {'entity_def_id': 'category_group',
  'permalink': 'lending-and-investments',
  'uuid': 'e03218e3-0279-3faa-0151-558385ce5b94',
  'value': 'Lending and Investments'},
 {'entity_def_id': 'category_group',
  'permalink': 'mobile-ec09',
  'uuid': 'ec09d1af-e88f-6a8d-1db8-1dd5e3d49ea0',
  'value': 'Mobile'}]

In [95]:
df["properties.categories"].values[0]

[{'entity_def_id': 'category',
  'permalink': 'banking',
  'uuid': '68250c3a-6cf5-86fb-281e-9072ae2d4fe8',
  'value': 'Banking'},
 {'entity_def_id': 'category',
  'permalink': 'finance',
  'uuid': 'ae5718dc-8602-c733-e62b-28e6167091ad',
  'value': 'Finance'},
 {'entity_def_id': 'category',
  'permalink': 'internet',
  'uuid': '2772a539-e636-09dc-30dc-b1a7deb98d0e',
  'value': 'Internet'},
 {'entity_def_id': 'category',
  'permalink': 'mobile',
  'uuid': 'e907cf2a-842d-8a30-a0b1-07d9de4a8178',
  'value': 'Mobile'},
 {'entity_def_id': 'category',
  'permalink': 'telecommunications',
  'uuid': 'e12cc7de-d276-007e-7d89-bdf3f8d1ffe7',
  'value': 'Telecommunications'}]